# Scenario 4. Robustness (regression design)

## Strategy 1. Measurement errors (Table 10)

In [1]:
R.version
library(tidyverse)
library(caret)
library(randomForest)
library(reticulate)
np <- import("numpy")

               _                           
platform       x86_64-pc-linux-gnu         
arch           x86_64                      
os             linux-gnu                   
system         x86_64, linux-gnu           
status                                     
major          3                           
minor          5.3                         
year           2019                        
month          03                          
day            11                          
svn rev        76217                       
language       R                           
version.string R version 3.5.3 (2019-03-11)
nickname       Great Truth                 

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──

✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   1.0.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift


randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:dplyr’:

    combine


The following object is masked from ‘package:ggplot2’:

    margin




### Configuration

In [2]:
data_path = "./data/simulation/s4"
path_genus = "./data/genus48"

y_path = sprintf('%s/%s', data_path, 'y.csv')
tree_info_path = './data/genus48/genus48_dic.csv'
count_path = './data/simulation/count_me'
count_list_path = './data/simulation/gcount_list.csv'
idx_path = './data/simulation/s1/idx.csv'

num_classes = 0 # regression
tree_level_list = c('Genus', 'Family', 'Order', 'Class', 'Phylum')

In [3]:
# # Read phylogenetic tree information

# phylogenetic_tree_info = read.csv(tree_info_path)
# phylogenetic_tree_info = phylogenetic_tree_info %>% select(tree_level_list)

# print(sprintf('Phylogenetic tree level list: %s', str_c(phylogenetic_tree_info %>% colnames, collapse = ', ')))

### Read dataset

#### Read training, test dataset

In [4]:
read_dataset <- function(x_path, y_path, sim){
    print(str_c('Load data for repetition ', sim))
    x = read.csv(x_path)
    y = read.csv(y_path)[,sim]
    x = (x - max(x)) / (max(x) - min(x))

    idxs = idxs_total[, sim]
    remain_idxs = setdiff(seq(1, dim(x)[1]), idxs)

    x_train = x[idxs,]
    x_test = x[remain_idxs,]
    y_train = y[idxs]
    y_test = y[remain_idxs]
    
    return (list(x_train, x_test, y_train, y_test))
}

In [5]:
idxs_total = read.csv(idx_path)
number_of_fold = dim(idxs_total)[2]; number_of_fold
x_list = read.csv(count_list_path, header = FALSE)
x_path = x_list$V1 %>% sprintf('%s/%s', count_path, .)

[1] 1000

#### Read true tree weight

In [6]:
tw_1 = np$load(sprintf('%s/tw_1.npy', data_path))

## Random Forest

### Importance type

* See <https://stats.stackexchange.com/questions/92419/relative-importance-of-a-set-of-predictors-in-a-random-forests-classification-in>

Here are the definitions of the variable importance measures.

- `type=1`: **Mean decrease in accuracy**
    - The first measure is computed from permuting Out-of-bag (OOB) data.
    - For each tree, the prediction error on the out-of-bag portion of the data is recorded (error rate for classification, MSE for regression). Then the same is done after permuting each predictor variable. 
    - The difference between the two are then averaged over all trees, and normalized by the standard deviation of the differences.
    - If the standard deviation of the differences is equal to 0 for a variable, the division is not done (but the average is almost always equal to 0 in that case).

- `type=2`: **Mean decrease in node impurity**
    - The second measure is the total decrease in node impurities from splitting on the variable, averaged over all trees. 
    - For classification, the node impurity is measured by the Gini index. 
    - For regression, it is measured by residual sum of squares.
    
### Feature selection

* `vi_f`: variable importance by Gini importance (`type=2`)
* `relative_vi_f` : relative variable importance
* `thrd`: threshold for relative variable importance
* Select features which have relative variable importance `relative_vi_f` equal or larger than threshold `thrd`

### Simulate for all $n$

In [7]:
random_forest_res <- function(fold, importance_type=2, fs_thrd = 0.1){
    print(sprintf('-----------------------------------------------------------------'))
    print(sprintf('Random Forest computation for %dth repetition', fold))

    dataset = read_dataset(x_path[fold], y_path, fold)
    x_train = dataset[[1]]
    x_test = dataset[[2]]
    y_train = dataset[[3]]
    y_test = dataset[[4]]

    fit.rf <- randomForest(y_train~.,data=x_train, ntree=1000,  mtry=10, importance=TRUE)
    train.pred <- fit.rf$predicted
    test.pred <- predict(fit.rf,x_test)

    train.mse <- mean((y_train - train.pred)^2)
    train.cor <- cor(y_train, train.pred)

    test.mse <- mean((y_test - test.pred)^2)
    test.cor <- cor(y_test, test.pred)
    
    # Feature selection
    ## variable importance
    vi_f = importance(fit.rf, type=importance_type)
    relative_vi_f <- vi_f / sum(vi_f)
    selected_genus <- ifelse(relative_vi_f >= fs_thrd, 1, 0)
    
    order <- order(relative_vi_f, decreasing = TRUE)
    sorted_relative_vi_f <- relative_vi_f[order]
    names(sorted_relative_vi_f) <- colnames(x_train)[order]
    print(sorted_relative_vi_f)

    fold_genus = apply(tw_1[fold,,], 1, sum)
    names(fold_genus) <- x_train %>% colnames

    fs_conf_table <- table(selected_genus, fold_genus)
    
    fs_sensitivity <- sensitivity(fs_conf_table) 
    fs_specificity <- specificity(fs_conf_table)
    fs_gmeasure <- sqrt(fs_sensitivity*fs_specificity)
    fs_accuracy <- sum(diag(fs_conf_table))/sum(fs_conf_table)

    print(sprintf('Train mse: %s, Train Correlation: %s', train.mse, train.cor))
    print(sprintf('Test mse: %s, Test Correlation: %s', test.mse, test.cor))
    print(sprintf('FS sensitivity: %s, FS sensitivity: %s, FS gmeasure: %s, FS accuracy: %s',
                  fs_sensitivity, fs_specificity, fs_gmeasure, fs_accuracy))
    
    return (c(train.mse, train.cor, test.mse, test.cor, fs_sensitivity, fs_specificity, fs_gmeasure, fs_accuracy))
}

In [8]:
set.seed(100)
# res <- sapply(seq(1,1), random_forest_res)
res <- sapply(seq(1,10), random_forest_res)
# res <- sapply(seq(1,number_of_fold), random_forest_res)

[1] "-----------------------------------------------------------------"
[1] "Random Forest computation for 1th repetition"
[1] "Load data for repetition 1"
                       Rothia                    Tropheryma 
                 0.2909319254                  0.1856261357 
                  Actinomyces             Propionibacterium 
                 0.0702610616                  0.0620667479 
               Pseudonocardia               Corynebacterium 
                 0.0358253198                  0.0247893868 
                    Atopobium     TM7_genera_incertae_sedis 
                 0.0180489952                  0.0164813458 
                Streptococcus                    Prevotella 
                 0.0135711852                  0.0131422695 
               Capnocytophaga                  Oribacterium 
                 0.0128482667                  0.0120665964 
                  Selenomonas                     Treponema 
                 0.0116877910                  0.01

In [9]:
results_table = res %>% t %>% data.frame
colnames(results_table) = c('Training MSE', 'Training Correlation', 'Test MSE', 'Test Correlation', 
                            'Taxa selection sensitivity','Taxa selection sensitivity',
                            'Taxa selection gmeasure', 'Taxa selection accuracy')
results_table

Training MSE,Training Correlation,Test MSE,Test Correlation,Taxa selection sensitivity,Taxa selection sensitivity,Taxa selection gmeasure,Taxa selection accuracy
0.7103146,0.9048869,0.8248670,0.8912506,1,0.1052632,0.3244428,0.6458333
0.7687064,0.8872230,0.7114735,0.9219048,1,0.1052632,0.3244428,0.6458333
0.7328649,0.9022667,0.7980956,0.8671227,1,0.1052632,0.3244428,0.6458333
0.8132727,0.8845817,0.6623301,0.8891512,1,0.1052632,0.3244428,0.6458333
0.7697988,0.9023046,0.8499341,0.8904468,1,0.1052632,0.3244428,0.6458333
0.6885309,0.9059698,0.7588928,0.8837913,1,0.1052632,0.3244428,0.6458333
0.7284909,0.8958500,0.7652187,0.8825066,1,0.1052632,0.3244428,0.6458333
0.7696044,0.8841988,0.8396656,0.8936129,1,0.1052632,0.3244428,0.6458333
0.7727862,0.8971588,0.7572901,0.8944676,1,0.1052632,0.3244428,0.6458333
0.7595324,0.8912674,0.7627159,0.8869221,1,0.1052632,0.3244428,0.6458333


In [10]:
print('Mean')
apply(results_table, 2, mean)

[1] "Mean"


Training MSE       Training Correlation 
                 0.7513902                  0.8955708 
                  Test MSE           Test Correlation 
                 0.7730483                  0.8901177 
Taxa selection sensitivity Taxa selection sensitivity 
                 1.0000000                  0.1052632 
   Taxa selection gmeasure    Taxa selection accuracy 
                 0.3244428                  0.6458333

In [11]:
print('SD')
apply(results_table, 2, sd)

[1] "SD"


Training MSE       Training Correlation 
                0.03625539                 0.00833999 
                  Test MSE           Test Correlation 
                0.05805033                 0.01366592 
Taxa selection sensitivity Taxa selection sensitivity 
                0.00000000                 0.00000000 
   Taxa selection gmeasure    Taxa selection accuracy 
                0.00000000                 0.00000000